import time
plc=1
time.sleep(1500*plc)         #run after 1 houre = 3600 seconds

In [1]:
import os
import glob
import scipy
import random
import bisect
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import signal
from scipy.signal import find_peaks
import matplotlib.pyplot as plt
from preprocess import *                         #ماژول استخراج پنجره ها
from data import *                          #ماژول محلی ورود داده ها
from augment import *                            #ماژول های داده افزایی
from lstm_cnn import *                           #ماژول های داده افزایی با lstm_cnn
import augment
import importlib
importlib.reload(augment)
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import BatchNormalization
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report,recall_score,precision_score
from tensorflow.keras import callbacks
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution2D,Conv1D,Dropout,MaxPooling1D,MaxPooling2D,Flatten,Dense
from tensorflow.keras import backend as K
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model
cwd = os.getcwd() #
fullscrn()

In [2]:
def maxindx(ar):
    ar_cpy=np.array(ar)
    pred=np.argmax(ar,axis=1)
    for i in range (len(pred)):
        if pred[i]==0:
            ar_cpy[i,0]=-1000
    pred=np.argmax(ar_cpy,axis=1)
    return(pred)

<h1><center><div style="direction:rtl;font-family:B Nazanin">Importing Data</div></center></h1>

In [3]:
cls_num=3
for i in range (1,cls_num):
    vars()['ecg'+str(i)],vars()['ecg_tst'+str(i)]=ECGFiveDays(i)
os.chdir(cwd)
clses_lens=np.array([])
i=0                               #جمع آوری داده ها و چاپ تعداد نمونه ی هر کلاس
ecg=np.array(ecg1)
print(1,len(vars()['ecg'+str(1)]), end='\t')
clses_lens=np.append(clses_lens,len(vars()['ecg'+str(1)]))
for i in range (2,cls_num):
    ecg=np.concatenate((ecg,vars()['ecg'+str(i)]),axis=0)
    clses_lens=np.append(clses_lens,len(vars()['ecg'+str(i)]))
    print( i,len(vars()['ecg'+str(i)]), end='\t')

mx_sig=max(clses_lens)
print('shape ecg=',np.shape(ecg))

first_train_shape= (23, 137)
first_test_shape= (861, 137)
classes_quantity= 2
tr_lbls=	 {1, 2}
Count_labels= [14  9]
max(train_feature_Altitude)= 5.4210751
min(train_feature_Altitude)= -6.5111593
first_train_sample=
 [ 1.0000000e+00  9.9133945e-03  3.3428888e-02 -5.7636009e-03
 -2.9279094e-02 -5.2794587e-02  9.9133945e-03 -2.1440596e-02
 -2.9279094e-02 -6.0633085e-02 -9.1987075e-02 -1.3602099e-02
 -6.0633085e-02 -1.3602099e-02 -7.6310081e-02 -6.0633085e-02
 -1.2334106e-01 -8.4148578e-02 -6.8471582e-02 -3.7117591e-02
 -7.6310081e-02 -6.0633085e-02 -1.2334106e-01 -6.0633085e-02
 -9.1987075e-02 -6.8471582e-02 -1.0766407e-01 -9.9825575e-02
 -1.4685656e-01 -2.9279094e-02  9.9133945e-03  9.6136870e-02
  1.2749086e-01  1.6668335e-01  2.9209932e-01  4.1751527e-01
  1.1965236e-01 -8.4148578e-02 -7.6310081e-02 -2.1740303e-01
 -1.9388755e-01 -2.7227252e-01 -2.4091853e-01 -3.1146501e-01
 -3.0362652e-01 -2.4091853e-01 -2.0956454e-01 -1.5469506e-01
 -2.1740303e-01 -9.1987075e-02  6.6050870e-01  2.83

C:\ProgramData\Anaconda3\lib\site-packages\pandas\util\_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


<h2><center><div style="direction:rtl;font-family:B Nazanin">Base Train windows</div></center></h1>

In [4]:
smpl_rte=480                                              # در ماژول ها نیز همین مقدار ثبت شده
i=0
windws=np.array([])

for cls in range (1,cls_num):                                 #ساخت پنجره های داده های آموزش اصلی
    vars()['wndws'+str(cls)]=np.array(vars()['ecg'+str(cls)])  
    
print("each class and its windows =")

cls_wndws=np.array([])
for cls in range (1,cls_num):                                # آرایه ی تعداد پنجره ی هر کلاس
    wns=len(vars()['wndws'+str(cls)])
    cls_wndws=np.append(cls_wndws,wns)
    print(cls, wns, end='\t')

mx_wndws=int(np.max(cls_wndws))                        # حداکثر تعداد پنجره ی موجود بین کلاس ها
print('\n max =', mx_wndws)

each class and its windows =
1 14	2 9	
 max = 14


In [5]:
#clses=3                                   #تعیین میزان افزایش نمونه (تولید داده)
mx_wndws=int(mx_wndws+6)  #mx_wndws*0.3  # =classes-1 برای بررسی الگوریتم تعداد کمی کلاس آزمایش شد
print(mx_wndws)

20


<h1><center><div style="direction:rtl;font-family:B Nazanin">Data Augmentation</div></center></h1>

In [6]:
for method in range(60,80,10):
    print(method)

60
70


In [7]:
rpt_cnn_clsfctn=3
rslt=np.array([])
rslts=np.array([])
for methd in range(30,40,10):
    for quality in range(1,7):
        if quality==1:
            sort=-5          #-5=Dis_sam_Cls=qlty1   #-4=var_Dis=qlty2    #-3=DisSm/DisOthr=qlty3    #-2=invrs_GDO_papr=qlty4   #-1=GDO_papr=qlty5  
            invers=0         #1=inverse sorting base of "sort"             #inverse of qlity3=qlty6  else=0
        elif quality==2:
            sort=-4          #-5=Dis_sam_Cls=qlty1   #-4=var_Dis=qlty2    #-3=DisSm/DisOthr=qlty3    #-2=invrs_GDO_papr=qlty4   #-1=GDO_papr=qlty5  
            invers=0         #1=inverse sorting base of "sort"             #inverse of qlity3=qlty6  else=0
        elif quality==3:
            sort=-3          #-5=Dis_sam_Cls=qlty1   #-4=var_Dis=qlty2    #-3=DisSm/DisOthr=qlty3    #-2=invrs_GDO_papr=qlty4   #-1=GDO_papr=qlty5  
            invers=0         #1=inverse sorting base of "sort"             #inverse of qlity3=qlty6  else=0
        elif quality==4:
            sort=-2          #-5=Dis_sam_Cls=qlty1   #-4=var_Dis=qlty2    #-3=DisSm/DisOthr=qlty3    #-2=invrs_GDO_papr=qlty4   #-1=GDO_papr=qlty5  
            invers=0         #1=inverse sorting base of "sort"             #inverse of qlity3=qlty6  else=0
        elif quality==5:
            sort=-1          #-5=Dis_sam_Cls=qlty1   #-4=var_Dis=qlty2    #-3=DisSm/DisOthr=qlty3    #-2=invrs_GDO_papr=qlty4   #-1=GDO_papr=qlty5  
            invers=0         #1=inverse sorting base of "sort"             #inverse of qlity3=qlty6  else=0
        else :
            sort=-3          #-5=Dis_sam_Cls=qlty1   #-4=var_Dis=qlty2    #-3=DisSm/DisOthr=qlty3    #-2=invrs_GDO_papr=qlty4   #-1=GDO_papr=qlty5  
            invers=1         #1=inverse sorting base of "sort"             #inverse of qlity3=qlty6  else=0
        for dp_slct_dstrb in range (1,6):#1-sequencial 2-linear 3-beta 4-exponential 5-unfrm
            rslt=np.array([])
            for repeat in range(1,rpt_cnn_clsfctn+1):
                for i in range (1,cls_num):                           #تعریف آرایه ی پنجره های کلاس ها
                    vars()['ecg_aug'+str(i)+str(methd)]=np.empty((0,smpl_rte+1), float) 
                for i in range (1,cls_num):                                     #تعریف آرایه ی پنجره های کلاس ها
                    vlum_win=mx_wndws-len(vars()['wndws'+str(i)])         #میزان داده افزایی در روش های عمیق (پنجره)
                    #print("vlum_win=",vlum_win)
                    vars()['ecg_aug'+str(i)+str(methd)]=np.append(vars()['ecg_aug'+str(i)+str(methd)],srtd_data(methd,i,vlum_win,dp_slct_dstrb,sort),axis=0)
                    #print('aug shape=',np.shape(scl_rndm(ecg,i,vlum_win,mag=.1)))
                    #print('generated data shape for class ', i, ' = ', np.shape(vars()['ecg_aug'+str(i)+str(methd)]))

                #add augmented data to base data
                xtrain=np.empty((0,smpl_rte+1), float) 
                for i in range (1,cls_num):   
                    xtrain=np.append(xtrain,vars()['wndws'+str(i)],axis=0)
                    xtrain=np.append(xtrain,vars()['ecg_aug'+str(i)+str(methd)],axis=0)
                    #print('Up to class ', i, 'train shape = ', np.shape(xtrain))

                # Normalization train windows
                mx_aug=np.max(xtrain[:,:-1])
                mx=np.max(ecg[:,:-1])
                mn_aug=np.min(xtrain[:,:-1])
                mn=np.min(ecg[:,:-1])
                xtrain[:,:-1]= 2*(xtrain[:,:-1]-mn_aug)/(mx_aug - mn_aug) - 1


                ###Test Windows:
                wndws_test=np.empty((0,len(wndws1[0])), float)
                i=0
                windws=np.array([])
                for cls in range (1,cls_num):                                 #ساخت پنجره های داده های آموزش اصلی
                    vars()['wndws_tst'+str(cls)]=np.empty((0,len(wndws1[0])), float)
                    vars()['wndws_tst'+str(cls)]=np.append(vars()['wndws_tst'+str(cls)],vars()['ecg_tst'+str(cls)],axis=0)   

                # Normalization test windows
                for i in range (1,cls_num):                
                    cls=i
                    #print('\n cls', i, ' >> ')
                    #print('max magnitude class', i , ' = ' ,np.max(vars()['wndws_tst'+str(cls)][:,:-1]))
                    #print('min magnitude class', i , ' = ' ,np.min(vars()['wndws_tst'+str(cls)][:,:-1]))
                    vars()['wndws_tst'+str(i)][:,:-1]= 2*(vars()['wndws_tst'+str(i)][:,:-1]-mn_aug)/(mx_aug - mn_aug) - 1
                    #print('after normalizing >>')
                    #print('max magnitude class', i , ' = ' ,np.max(vars()['wndws_tst'+str(cls)][:,:-1]))
                    #print('min magnitude class', i , ' = ' ,np.min(vars()['wndws_tst'+str(cls)][:,:-1]))
                #print("each class and its windows =")                    #تجمیع کلاس های تست
                cls_wndws=np.array([])
                for cls in range (1,cls_num):                                # آرایه ی تعداد پنجره ی هر کلاس
                    wns=len(vars()['wndws_tst'+str(cls)])
                    cls_wndws=np.append(cls_wndws,wns)
                    #print(cls, wns, end='\t')
                    wndws_test=np.append(wndws_test,vars()['wndws_tst'+str(cls)],axis=0)
                mx_wndws=int(np.max(cls_wndws))                        # حداکثر تعداد پنجره ی موجود بین کلاس ها
                #print('\n max instance in classes of test windows =', mx_wndws)

                #print('min tst =', np.min(wndws_test[:,:-1]))
                #print('max tst =', np.max(wndws_test[:,:-1]))

                Xtest=wndws_test[:,:-1]
                ytest=np.int16(wndws_test[:,-1])

                #fullprint(ytest)

                trainx=np.random.permutation(xtrain)
                Xtrain=np.array(trainx[:,:-1])
                ytrain=np.int16(trainx[:,-1])

                X_train=np.array(Xtrain)
                y_train=np.array(ytrain)
                X_test=np.array(Xtest)
                y_test=np.array(ytest)
                X_valid=np.array(Xtrain)
                y_valid=np.array(ytrain)
                #print('X_train => ', X_train.shape)
                #print('y_train => ', y_train.shape)
                #print('X_test  => ', X_test.shape)
                #print('y_test  => ', y_test.shape)
                #print('X_valid  => ', X_valid.shape)
                #print('y_valid  => ', y_valid.shape)

                y_train_cat=to_categorical(y_train)
                y_valid_cat=to_categorical(y_valid)
                y_test_cat=to_categorical(y_test)
                X_train = np.expand_dims(X_train, axis=2)
                X_valid = np.expand_dims(X_valid, axis=2)
                X_test = np.expand_dims(X_test, axis=2)

                ####### NETWORK #######
                accuracy=0
                acc_crs=np.array([])
                ##for i in range (10):
                ##    if accuracy<0.2 :
                verbose, epochs, batch_size = 0, 1000, 10
                n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train_cat.shape[1]
                steps_per_epoch = len(X_train)//batch_size
                validation_steps = len(X_valid)//batch_size # if you have test data
                model_crs = Sequential()
                BatchNormalization()
                model_crs.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(n_timesteps,n_features)))
                #model_crs.add(MaxPooling1D(pool_size=3))
                BatchNormalization()
                model_crs.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
                #model_crs.add(Dropout(0.5))
                model_crs.add(MaxPooling1D(pool_size=3))
                BatchNormalization()
                model_crs.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
                model_crs.add(MaxPooling1D(pool_size=3))
                BatchNormalization()
                model_crs.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
                model_crs.add(MaxPooling1D(pool_size=3))
                BatchNormalization()
                model_crs.add(Conv1D(filters=256, kernel_size=3, activation='relu'))
                model_crs.add(MaxPooling1D(pool_size=3))
                '''model_crs.add(Conv1D(filters=512, kernel_size=3, activation='relu'))
                model_crs.add(MaxPooling1D(pool_size=3))
                model_crs.add(Conv1D(filters=1024, kernel_size=3, activation='relu'))
                model_crs.add(MaxPooling1D(pool_size=3))
                '''
                model_crs.add(Flatten())
                #model_crs.add(Dropout(0.5))
                model_crs.add(Dense(2000, activation='relu'))
                model_crs.add(Dense(n_outputs, activation='softmax'))
                model_crs.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
                # fit network   #CategoricalCrossentropy #sparse_categorical_crossentropy #SparseCategoricalCrossentropy
                earlystopping = callbacks.EarlyStopping(monitor ="val_accuracy", mode ="max", patience = 20, restore_best_weights = True)
                model_crs.fit(X_train, y_train_cat, epochs=epochs, batch_size=batch_size, verbose=verbose, validation_data = (X_valid, y_valid_cat), callbacks =[earlystopping])
                # evaluate model_crs
                _, accuracy = model_crs.evaluate(X_valid, y_valid_cat, batch_size=batch_size, verbose=verbose)
                #print('np.shape(X_test)=',np.shape(X_test))
                #model_crs.summary()
                predict_x=model_crs.predict(X_test)              # Function 1
                y_pred_crs=maxindx(predict_x)                    # function from augment.py to remove 0 index predictions
                rprt=classification_report(y_test, y_pred_crs,output_dict=True)
                rslt=np.append(rslt,rprt['macro avg']['f1-score'])
            rslts=np.append(rslts,['mthd '+str(methd)+'  q '+str(quality)+'  Dist '+str(dp_slct_dstrb)+'  average= '+str(int(100*np.mean(rslt)))+'  max= '+str(int(100*np.max(rslt)))+'  var= '+str(int(np.var(100*rslt)))],axis=0)

27/27 [==============================] - 0s 2ms/step


In [8]:
rslts

array(['mthd 60  q 1  Dist 1  average= 92  max= 96  var= 7',
       'mthd 60  q 1  Dist 2  average= 89  max= 91  var= 4',
       'mthd 60  q 1  Dist 3  average= 88  max= 91  var= 9',
       'mthd 60  q 1  Dist 4  average= 83  max= 88  var= 10',
       'mthd 60  q 1  Dist 5  average= 89  max= 91  var= 5',
       'mthd 60  q 2  Dist 1  average= 84  max= 88  var= 18',
       'mthd 60  q 2  Dist 2  average= 90  max= 91  var= 0',
       'mthd 60  q 2  Dist 3  average= 93  max= 96  var= 5',
       'mthd 60  q 2  Dist 4  average= 90  max= 93  var= 18',
       'mthd 60  q 2  Dist 5  average= 86  max= 90  var= 10',
       'mthd 60  q 3  Dist 1  average= 85  max= 89  var= 19',
       'mthd 60  q 3  Dist 2  average= 89  max= 94  var= 18',
       'mthd 60  q 3  Dist 3  average= 88  max= 93  var= 28',
       'mthd 60  q 3  Dist 4  average= 88  max= 97  var= 36',
       'mthd 60  q 3  Dist 5  average= 85  max= 91  var= 16',
       'mthd 60  q 4  Dist 1  average= 88  max= 94  var= 38',
       'mthd 6

In [ ]:
rpt_cnn_clsfctn=3
rslt=np.array([])
rslts=np.array([])
for methd in range(30,40,10):
    for quality in range(1,7):
        if quality==1:
            sort=-5          #-5=Dis_sam_Cls=qlty1   #-4=var_Dis=qlty2    #-3=DisSm/DisOthr=qlty3    #-2=invrs_GDO_papr=qlty4   #-1=GDO_papr=qlty5  
            invers=0         #1=inverse sorting base of "sort"             #inverse of qlity3=qlty6  else=0
        elif quality==2:
            sort=-4          #-5=Dis_sam_Cls=qlty1   #-4=var_Dis=qlty2    #-3=DisSm/DisOthr=qlty3    #-2=invrs_GDO_papr=qlty4   #-1=GDO_papr=qlty5  
            invers=0         #1=inverse sorting base of "sort"             #inverse of qlity3=qlty6  else=0
        elif quality==3:
            sort=-3          #-5=Dis_sam_Cls=qlty1   #-4=var_Dis=qlty2    #-3=DisSm/DisOthr=qlty3    #-2=invrs_GDO_papr=qlty4   #-1=GDO_papr=qlty5  
            invers=0         #1=inverse sorting base of "sort"             #inverse of qlity3=qlty6  else=0
        elif quality==4:
            sort=-2          #-5=Dis_sam_Cls=qlty1   #-4=var_Dis=qlty2    #-3=DisSm/DisOthr=qlty3    #-2=invrs_GDO_papr=qlty4   #-1=GDO_papr=qlty5  
            invers=0         #1=inverse sorting base of "sort"             #inverse of qlity3=qlty6  else=0
        elif quality==5:
            sort=-1          #-5=Dis_sam_Cls=qlty1   #-4=var_Dis=qlty2    #-3=DisSm/DisOthr=qlty3    #-2=invrs_GDO_papr=qlty4   #-1=GDO_papr=qlty5  
            invers=0         #1=inverse sorting base of "sort"             #inverse of qlity3=qlty6  else=0
        else :
            sort=-3          #-5=Dis_sam_Cls=qlty1   #-4=var_Dis=qlty2    #-3=DisSm/DisOthr=qlty3    #-2=invrs_GDO_papr=qlty4   #-1=GDO_papr=qlty5  
            invers=1         #1=inverse sorting base of "sort"             #inverse of qlity3=qlty6  else=0
        for dp_slct_dstrb in range (1,6):#1-sequencial 2-linear 3-beta 4-exponential 5-unfrm
            rslt=np.array([])
            for repeat in range(1,rpt_cnn_clsfctn+1):
                for i in range (1,cls_num):                           #تعریف آرایه ی پنجره های کلاس ها
                    vars()['ecg_aug'+str(i)+str(methd)]=np.empty((0,smpl_rte+1), float) 
                for i in range (1,cls_num):                                     #تعریف آرایه ی پنجره های کلاس ها
                    vlum_win=mx_wndws-len(vars()['wndws'+str(i)])         #میزان داده افزایی در روش های عمیق (پنجره)
                    #print("vlum_win=",vlum_win)
                    vars()['ecg_aug'+str(i)+str(methd)]=np.append(vars()['ecg_aug'+str(i)+str(methd)],srtd_data(methd,i,vlum_win,dp_slct_dstrb,sort),axis=0)
                    #print('aug shape=',np.shape(scl_rndm(ecg,i,vlum_win,mag=.1)))
                    #print('generated data shape for class ', i, ' = ', np.shape(vars()['ecg_aug'+str(i)+str(methd)]))

                #add augmented data to base data
                xtrain=np.empty((0,smpl_rte+1), float) 
                for i in range (1,cls_num):   
                    xtrain=np.append(xtrain,vars()['wndws'+str(i)],axis=0)
                    xtrain=np.append(xtrain,vars()['ecg_aug'+str(i)+str(methd)],axis=0)
                    #print('Up to class ', i, 'train shape = ', np.shape(xtrain))

                # Normalization train windows
                mx_aug=np.max(xtrain[:,:-1])
                mx=np.max(ecg[:,:-1])
                mn_aug=np.min(xtrain[:,:-1])
                mn=np.min(ecg[:,:-1])
                xtrain[:,:-1]= 2*(xtrain[:,:-1]-mn_aug)/(mx_aug - mn_aug) - 1


                ###Test Windows:
                wndws_test=np.empty((0,len(wndws1[0])), float)
                i=0
                windws=np.array([])
                for cls in range (1,cls_num):                                 #ساخت پنجره های داده های آموزش اصلی
                    vars()['wndws_tst'+str(cls)]=np.empty((0,len(wndws1[0])), float)
                    vars()['wndws_tst'+str(cls)]=np.append(vars()['wndws_tst'+str(cls)],vars()['ecg_tst'+str(cls)],axis=0)   

                # Normalization test windows
                for i in range (1,cls_num):                
                    cls=i
                    #print('\n cls', i, ' >> ')
                    #print('max magnitude class', i , ' = ' ,np.max(vars()['wndws_tst'+str(cls)][:,:-1]))
                    #print('min magnitude class', i , ' = ' ,np.min(vars()['wndws_tst'+str(cls)][:,:-1]))
                    vars()['wndws_tst'+str(i)][:,:-1]= 2*(vars()['wndws_tst'+str(i)][:,:-1]-mn_aug)/(mx_aug - mn_aug) - 1
                    #print('after normalizing >>')
                    #print('max magnitude class', i , ' = ' ,np.max(vars()['wndws_tst'+str(cls)][:,:-1]))
                    #print('min magnitude class', i , ' = ' ,np.min(vars()['wndws_tst'+str(cls)][:,:-1]))
                #print("each class and its windows =")                    #تجمیع کلاس های تست
                cls_wndws=np.array([])
                for cls in range (1,cls_num):                                # آرایه ی تعداد پنجره ی هر کلاس
                    wns=len(vars()['wndws_tst'+str(cls)])
                    cls_wndws=np.append(cls_wndws,wns)
                    #print(cls, wns, end='\t')
                    wndws_test=np.append(wndws_test,vars()['wndws_tst'+str(cls)],axis=0)
                mx_wndws=int(np.max(cls_wndws))                        # حداکثر تعداد پنجره ی موجود بین کلاس ها
                #print('\n max instance in classes of test windows =', mx_wndws)

                #print('min tst =', np.min(wndws_test[:,:-1]))
                #print('max tst =', np.max(wndws_test[:,:-1]))

                Xtest=wndws_test[:,:-1]
                ytest=np.int16(wndws_test[:,-1])

                #fullprint(ytest)

                trainx=np.random.permutation(xtrain)
                Xtrain=np.array(trainx[:,:-1])
                ytrain=np.int16(trainx[:,-1])

                X_train=np.array(Xtrain)
                y_train=np.array(ytrain)
                X_test=np.array(Xtest)
                y_test=np.array(ytest)
                X_valid=np.array(Xtrain)
                y_valid=np.array(ytrain)
                #print('X_train => ', X_train.shape)
                #print('y_train => ', y_train.shape)
                #print('X_test  => ', X_test.shape)
                #print('y_test  => ', y_test.shape)
                #print('X_valid  => ', X_valid.shape)
                #print('y_valid  => ', y_valid.shape)

                y_train_cat=to_categorical(y_train)
                y_valid_cat=to_categorical(y_valid)
                y_test_cat=to_categorical(y_test)
                X_train = np.expand_dims(X_train, axis=2)
                X_valid = np.expand_dims(X_valid, axis=2)
                X_test = np.expand_dims(X_test, axis=2)

                ####### NETWORK #######
                accuracy=0
                acc_crs=np.array([])
                ##for i in range (10):
                ##    if accuracy<0.2 :
                verbose, epochs, batch_size = 0, 1000, 10
                n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train_cat.shape[1]
                steps_per_epoch = len(X_train)//batch_size
                validation_steps = len(X_valid)//batch_size # if you have test data
                model_crs = Sequential()
                BatchNormalization()
                model_crs.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(n_timesteps,n_features)))
                #model_crs.add(MaxPooling1D(pool_size=3))
                BatchNormalization()
                model_crs.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
                #model_crs.add(Dropout(0.5))
                model_crs.add(MaxPooling1D(pool_size=3))
                BatchNormalization()
                model_crs.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
                model_crs.add(MaxPooling1D(pool_size=3))
                BatchNormalization()
                model_crs.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
                model_crs.add(MaxPooling1D(pool_size=3))
                BatchNormalization()
                model_crs.add(Conv1D(filters=256, kernel_size=3, activation='relu'))
                model_crs.add(MaxPooling1D(pool_size=3))
                '''model_crs.add(Conv1D(filters=512, kernel_size=3, activation='relu'))
                model_crs.add(MaxPooling1D(pool_size=3))
                model_crs.add(Conv1D(filters=1024, kernel_size=3, activation='relu'))
                model_crs.add(MaxPooling1D(pool_size=3))
                '''
                model_crs.add(Flatten())
                #model_crs.add(Dropout(0.5))
                model_crs.add(Dense(2000, activation='relu'))
                model_crs.add(Dense(n_outputs, activation='softmax'))
                model_crs.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
                # fit network   #CategoricalCrossentropy #sparse_categorical_crossentropy #SparseCategoricalCrossentropy
                earlystopping = callbacks.EarlyStopping(monitor ="val_accuracy", mode ="max", patience = 20, restore_best_weights = True)
                model_crs.fit(X_train, y_train_cat, epochs=epochs, batch_size=batch_size, verbose=verbose, validation_data = (X_valid, y_valid_cat), callbacks =[earlystopping])
                # evaluate model_crs
                _, accuracy = model_crs.evaluate(X_valid, y_valid_cat, batch_size=batch_size, verbose=verbose)
                #print('np.shape(X_test)=',np.shape(X_test))
                #model_crs.summary()
                predict_x=model_crs.predict(X_test)              # Function 1
                y_pred_crs=maxindx(predict_x)                    # function from augment.py to remove 0 index predictions
                rprt=classification_report(y_test, y_pred_crs,output_dict=True)
                rslt=np.append(rslt,rprt['macro avg']['f1-score'])
            rslts=np.append(rslts,['mthd '+str(methd)+'  q '+str(quality)+'  Dist '+str(dp_slct_dstrb)+'  average= '+str(int(100*np.mean(rslt)))+'  max= '+str(int(100*np.max(rslt)))+'  var= '+str(int(np.var(100*rslt)))],axis=0)